In [21]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os
import csv
import re
import pandas as pd
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import numpy as np
from tqdm import tqdm

In [2]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/buckycheng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
sia = SentimentIntensityAnalyzer()

In [4]:
pd.read_csv('Sentiments/stock_lexicon.csv')

,Item,POS,Aff_Score,Neg_Score
0,'em,PR,0.379542,0.533419
1,'n,CC,1.413176,1.199917
2,'n handle,NaN,2.839000,2.941000
3,'s a,NaN,-0.023077,-0.013615
4,'s abc,NaN,2.261667,2.300000
...,...,...,...,...
20545,zoom,VB,-0.652273,-0.649174
20546,zortrades.com,NN,2.141000,2.163000
20547,zte,NN,4.934000,5.084000
20548,zuck,NN,-0.237857,-0.185172


In [5]:
stock_lexicon=pd.read_csv('Sentiments/stock_lexicon.csv')
positive_data=pd.read_csv('Sentiments/positive_data.csv')
negative_data=pd.read_csv('Sentiments/negative_data.csv')

In [6]:
positive =[]
for i in positive_data['Word'].values:
    positive.append(i.strip())

negative =[]
for i in negative_data['word'].values:
    negative.append(i)


In [7]:
stock_lexicon['sentiment']=(stock_lexicon['Aff_Score']+stock_lexicon['Neg_Score'])/2
stock_lexicon = dict(zip(stock_lexicon.Item, stock_lexicon.sentiment))
stock_lexicon = {h:t for h,t in stock_lexicon.items() if len(h.split(' '))==1}
stock_lex_scaled = {}
for h, t in stock_lexicon.items():
    if t > 0:
        stock_lex_scaled[h] = t / max(stock_lexicon.values()) * 4
    else:
        stock_lex_scaled[h] = t / min(stock_lexicon.values()) * -4
print(max(stock_lexicon.values()))

5.009


In [8]:
final_lexicon = {}
final_lexicon.update({word:2.0 for word in positive})
final_lexicon.update({word:-2.0 for word in negative})
final_lexicon.update(stock_lex_scaled)
final_lexicon.update(sia.lexicon)
sia.lexicon = final_lexicon

In [9]:
def sentiment_scores(sentence):
    sent = sia.polarity_scores(sentence)
    return format(str(sent))

In [17]:
#clean_data, file_name
def data_sentiment(path, processingFile, cik, ticker, fdate, form):
    html_regex = re.compile(r'<.*?>')
    senti_data=[]
    
    FilePath = os.path.join(path, processingFile)
    print("Now File: " + FilePath)
    resultdict = dict()
    resultdict['CIK'] = cik
    resultdict['CONAME'] = ticker
    resultdict['FDATE'] = fdate
    resultdict['FORM'] = form
    resultdict['FileName'] = processingFile
                
    with open(FilePath, 'r' , errors="replace") as in_file:
        content = in_file.read().lower()
        content = re.sub(html_regex,'',content) 
        content = content.replace('&nbsp;','') 
        content = re.sub(r'&#\d+;', '', content)
        content = re.sub(r'[^\x00-\x7f]', ' ', content)
        content = re.sub(' +',' ', content)
        if content:
            content = str(content).replace('\n', '')
            #make tokens  
            tokens = word_tokenize(content)
            text = nltk.Text(tokens)  
            
            
            #batch size 2000 words
            start_point=0
            batch_size=2000
            total_num = int(len(tokens)/2000)
    
            
            
            negscore_array = np.array([])
            neuscore_array = np.array([])
            posscore_array = np.array([])
            
            for i in range(total_num):
                first_list = tokens[start_point:start_point+batch_size]
                first_list_str = " ".join(first_list)
                #get sentiment scores
                sent_scores = sentiment_scores(first_list_str)
                start_point = start_point+batch_size 

                #convert each score to numpy array
                neg_scores = float(sent_scores.split(",")[0].split(":")[1])
                neu_scores = float(sent_scores.split(",")[1].split(":")[1])
                pos_scores = float(sent_scores.split(",")[2].split(":")[1])

                negscore_array = np.append(negscore_array, neg_scores)
                neuscore_array = np.append(neuscore_array, neu_scores)
                posscore_array = np.append(posscore_array, pos_scores)   

            neg_meanscore = np.mean(negscore_array)
            neu_meanscore = np.mean(neuscore_array)
            pos_meanscore = np.mean(posscore_array)

            resultdict['neg_score'] = neg_meanscore
            resultdict['neu_score'] = neu_meanscore
            resultdict['pos_score'] = pos_meanscore
            
            print("get scores")
            print(resultdict)
            exit
        senti_data.append(resultdict)
        in_file.close()
    return senti_data

In [11]:
df_trickers=pd.read_csv('AllTickers.csv')



In [12]:
df_trickers=df_trickers[:50]
df_trickers.head().append(df_trickers.tail())

,Company,CIK
0,3M Company,66740
1,Abbott Laboratories,1800
2,AbbVie Inc.,1551152
3,ABIOMED Inc,815094
4,Accenture plc,1467373
5,Activision Blizzard,718877
6,Adobe Inc.,796343
7,Advanced Micro Devices Inc,2488
8,Advance Auto Parts,1158449
9,AES Corp,874761


## 10-K sentiment analysis 

In [19]:
path = '/Users/buckycheng/Jupyter/Huge_Stock/10K'

senti_path_10K = '/Users/buckycheng/Jupyter/Huge_Stock/Sentiments/10K'

for i in range(len(df_trickers)):
    cik=df_trickers['CIK'].iloc[i]
    ticker=df_trickers['Company'].iloc[i]
    CIK_Folder = path+'/'+str(cik)+'/clean_data'
    os.chdir(CIK_Folder)
    file_list = [fname for fname in os.listdir()]
    for cik_file in file_list:
        fname = cik_file.split('.')[0]
        fdate = fname.split('_')[1]
        ftype = '10-K'
        file_path=cik_file
        
        senti_list = data_sentiment(CIK_Folder, file_path, cik, ticker, fdate, ftype)
        result_df = pd.DataFrame(senti_list)
       # headers are CIK	CONAME	FDATE	FORM	SECFNAME	neg_score	neu_score	pos_score
        result_df.to_csv(senti_path_10K + '/' +str(cik)+'_'+'10-K_sentiment'+'.csv',encoding='utf-8', mode = 'a', index=False, header=False)
        print(str(cik)+' finished')

Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/66740/clean_data/66740_2007-02-26.txt
get scores
{'CIK': 66740, 'CONAME': '3M Company', 'FDATE': '2007-02-26', 'FORM': '10-K', 'FileName': '66740_2007-02-26.txt', 'neg_score': 0.26053571428571426, 'neu_score': 0.3734285714285714, 'pos_score': 0.3660714285714285}
66740 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/66740/clean_data/66740_2018-02-08.txt
get scores
{'CIK': 66740, 'CONAME': '3M Company', 'FDATE': '2018-02-08', 'FORM': '10-K', 'FileName': '66740_2018-02-08.txt', 'neg_score': 0.26641860465116274, 'neu_score': 0.3642558139534884, 'pos_score': 0.3693720930232558}
66740 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/66740/clean_data/66740_2009-02-17.txt
get scores
{'CIK': 66740, 'CONAME': '3M Company', 'FDATE': '2009-02-17', 'FORM': '10-K', 'FileName': '66740_2009-02-17.txt', 'neg_score': 0.2656388888888888, 'neu_score': 0.36938888888888893, 'pos_score': 0.3650277777777778}
66740 finished
Now File: /Us

get scores
{'CIK': 1800, 'CONAME': 'Abbott Laboratories', 'FDATE': '2004-02-25', 'FORM': '10-K', 'FileName': '1800_2004-02-25.txt', 'neg_score': 0.26465, 'neu_score': 0.3839, 'pos_score': 0.35135}
1800 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/1800/clean_data/1800_2010-02-19.txt
get scores
{'CIK': 1800, 'CONAME': 'Abbott Laboratories', 'FDATE': '2010-02-19', 'FORM': '10-K', 'FileName': '1800_2010-02-19.txt', 'neg_score': 0.263, 'neu_score': 0.37731818181818183, 'pos_score': 0.35959090909090907}
1800 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/1800/clean_data/1800_2013-02-15.txt
get scores
{'CIK': 1800, 'CONAME': 'Abbott Laboratories', 'FDATE': '2013-02-15', 'FORM': '10-K', 'FileName': '1800_2013-02-15.txt', 'neg_score': 0.2695416666666666, 'neu_score': 0.36633333333333334, 'pos_score': 0.3640833333333333}
1800 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/1800/clean_data/1800_2005-03-02.txt
get scores
{'CIK': 1800, 'CONAME': 'Abbott Labor

get scores
{'CIK': 815094, 'CONAME': 'ABIOMED Inc', 'FDATE': '2006-06-14', 'FORM': '10-K', 'FileName': '815094_2006-06-14.txt', 'neg_score': 0.273, 'neu_score': 0.3351875, 'pos_score': 0.39175}
815094 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/815094/clean_data/815094_2020-05-21.txt
get scores
{'CIK': 815094, 'CONAME': 'ABIOMED Inc', 'FDATE': '2020-05-21', 'FORM': '10-K', 'FileName': '815094_2020-05-21.txt', 'neg_score': 0.27515625, 'neu_score': 0.34471875, 'pos_score': 0.38025}
815094 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/815094/clean_data/815094_2007-06-13.txt
get scores
{'CIK': 815094, 'CONAME': 'ABIOMED Inc', 'FDATE': '2007-06-13', 'FORM': '10-K', 'FileName': '815094_2007-06-13.txt', 'neg_score': 0.271875, 'neu_score': 0.3280416666666667, 'pos_score': 0.40004166666666663}
815094 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/815094/clean_data/815094_2001-06-28.txt
get scores
{'CIK': 815094, 'CONAME': 'ABIOMED Inc', 'FDATE': '2001-

get scores
{'CIK': 1467373, 'CONAME': 'Accenture plc', 'FDATE': '2014-10-24', 'FORM': '10-K', 'FileName': '1467373_2014-10-24.txt', 'neg_score': 0.2715714285714285, 'neu_score': 0.3371071428571427, 'pos_score': 0.3913214285714286}
1467373 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/1467373/clean_data/1467373_2016-10-28.txt
get scores
{'CIK': 1467373, 'CONAME': 'Accenture plc', 'FDATE': '2016-10-28', 'FORM': '10-K', 'FileName': '1467373_2016-10-28.txt', 'neg_score': 0.2745185185185185, 'neu_score': 0.3363703703703703, 'pos_score': 0.3891481481481482}
1467373 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/1467373/clean_data/1467373_2015-10-30.txt
get scores
{'CIK': 1467373, 'CONAME': 'Accenture plc', 'FDATE': '2015-10-30', 'FORM': '10-K', 'FileName': '1467373_2015-10-30.txt', 'neg_score': 0.2732307692307693, 'neu_score': 0.3346538461538462, 'pos_score': 0.3922692307692308}
1467373 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/1467373/clean_data/

get scores
{'CIK': 796343, 'CONAME': 'Adobe Inc.', 'FDATE': '2003-02-26', 'FORM': '10-K', 'FileName': '796343_2003-02-26.txt', 'neg_score': 0.2597714285714286, 'neu_score': 0.3932285714285714, 'pos_score': 0.3468571428571429}
796343 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/796343/clean_data/796343_2015-01-20.txt
get scores
{'CIK': 796343, 'CONAME': 'Adobe Inc.', 'FDATE': '2015-01-20', 'FORM': '10-K', 'FileName': '796343_2015-01-20.txt', 'neg_score': 0.27329032258064523, 'neu_score': 0.3315161290322581, 'pos_score': 0.3950967741935485}
796343 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/796343/clean_data/796343_2020-01-21.txt
get scores
{'CIK': 796343, 'CONAME': 'Adobe Inc.', 'FDATE': '2020-01-21', 'FORM': '10-K', 'FileName': '796343_2020-01-21.txt', 'neg_score': 0.2596969696969697, 'neu_score': 0.363969696969697, 'pos_score': 0.3763636363636364}
796343 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/796343/clean_data/796343_2018-01-22.txt
g

...

get scores
{'CIK': 91142, 'CONAME': 'A.O. Smith Corp', 'FDATE': '2003-02-20', 'FORM': '10-K', 'FileName': '91142_2003-02-20.txt', 'neg_score': 0.2283571428571429, 'neu_score': 0.4883571428571428, 'pos_score': 0.28328571428571425}
91142 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/91142/clean_data/91142_2008-02-20.txt
get scores
{'CIK': 91142, 'CONAME': 'A.O. Smith Corp', 'FDATE': '2008-02-20', 'FORM': '10-K', 'FileName': '91142_2008-02-20.txt', 'neg_score': 0.284, 'neu_score': 0.36128571428571427, 'pos_score': 0.3545714285714286}
91142 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/6769/clean_data/6769_2004-03-12.txt
get scores
{'CIK': 6769, 'CONAME': 'Apache Corporation', 'FDATE': '2004-03-12', 'FORM': '10-K', 'FileName': '6769_2004-03-12.txt', 'neg_score': 0.2304871794871795, 'neu_score': 0.45112820512820506, 'pos_score': 0.3183589743589744}
6769 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/6769/clean_data/6769_2014-02-28.txt
get scores
{'CI

get scores
{'CIK': 922864, 'CONAME': 'Apartment Investment & Management', 'FDATE': '2002-03-20', 'FORM': '10-K', 'FileName': '922864_2002-03-20.txt', 'neg_score': 0.2622, 'neu_score': 0.35696, 'pos_score': 0.3806399999999999}
922864 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/922864/clean_data/922864_2008-02-29.txt
get scores
{'CIK': 922864, 'CONAME': 'Apartment Investment & Management', 'FDATE': '2008-02-29', 'FORM': '10-K', 'FileName': '922864_2008-02-29.txt', 'neg_score': 0.21915789473684213, 'neu_score': 0.4526578947368421, 'pos_score': 0.3280526315789474}
922864 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/922864/clean_data/922864_2007-03-01.txt
get scores
{'CIK': 922864, 'CONAME': 'Apartment Investment & Management', 'FDATE': '2007-03-01', 'FORM': '10-K', 'FileName': '922864_2007-03-01.txt', 'neg_score': 0.21783333333333332, 'neu_score': 0.4616944444444444, 'pos_score': 0.32058333333333333}
922864 finished
Now File: /Users/buckycheng/Jupyter/Huge_St

get scores
{'CIK': 320193, 'CONAME': 'Apple Inc.', 'FDATE': '2003-12-19', 'FORM': '10-K', 'FileName': '320193_2003-12-19.txt', 'neg_score': 0.30248484848484847, 'neu_score': 0.32106060606060605, 'pos_score': 0.37636363636363634}
320193 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/320193/clean_data/320193_2011-10-26.txt
get scores
{'CIK': 320193, 'CONAME': 'Apple Inc.', 'FDATE': '2011-10-26', 'FORM': '10-K', 'FileName': '320193_2011-10-26.txt', 'neg_score': 0.2808260869565217, 'neu_score': 0.3169565217391304, 'pos_score': 0.4022608695652174}
320193 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/320193/clean_data/320193_2006-12-29.txt
get scores
{'CIK': 320193, 'CONAME': 'Apple Inc.', 'FDATE': '2006-12-29', 'FORM': '10-K', 'FileName': '320193_2006-12-29.txt', 'neg_score': 0.2759230769230769, 'neu_score': 0.34538461538461546, 'pos_score': 0.3786923076923078}
320193 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/320193/clean_data/320193_2000-12-14.t

get scores
{'CIK': 6951, 'CONAME': 'Applied Materials Inc.', 'FDATE': '2000-01-31', 'FORM': '10-K', 'FileName': '6951_2000-01-31.txt', 'neg_score': 0.2283333333333333, 'neu_score': 0.4431666666666667, 'pos_score': 0.3283333333333333}
6951 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/6951/clean_data/6951_2018-12-13.txt
get scores
{'CIK': 6951, 'CONAME': 'Applied Materials Inc.', 'FDATE': '2018-12-13', 'FORM': '10-K', 'FileName': '6951_2018-12-13.txt', 'neg_score': 0.27986363636363637, 'neu_score': 0.3437727272727273, 'pos_score': 0.3763636363636364}
6951 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/6951/clean_data/6951_2013-12-04.txt
get scores
{'CIK': 6951, 'CONAME': 'Applied Materials Inc.', 'FDATE': '2013-12-04', 'FORM': '10-K', 'FileName': '6951_2013-12-04.txt', 'neg_score': 0.2765925925925926, 'neu_score': 0.3581481481481482, 'pos_score': 0.3652962962962963}
6951 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/6951/clean_data/6951_2006-12-1

get scores
{'CIK': 7084, 'CONAME': 'Archer-Daniels-Midland Co', 'FDATE': '2001-09-28', 'FORM': '10-K', 'FileName': '7084_2001-09-28.txt', 'neg_score': 0.24714285714285714, 'neu_score': 0.41714285714285715, 'pos_score': 0.33571428571428574}
7084 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/7084/clean_data/7084_2015-02-20.txt
get scores
{'CIK': 7084, 'CONAME': 'Archer-Daniels-Midland Co', 'FDATE': '2015-02-20', 'FORM': '10-K', 'FileName': '7084_2015-02-20.txt', 'neg_score': 0.28438461538461535, 'neu_score': 0.3505, 'pos_score': 0.36519230769230765}
7084 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/7084/clean_data/7084_2012-08-27.txt
get scores
{'CIK': 7084, 'CONAME': 'Archer-Daniels-Midland Co', 'FDATE': '2012-08-27', 'FORM': '10-K', 'FileName': '7084_2012-08-27.txt', 'neg_score': 0.27136000000000005, 'neu_score': 0.36719999999999997, 'pos_score': 0.36132000000000003}
7084 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10K/7084/clean_data/7084_2008-

# 10-Q sentiment analysis

In [20]:
path = '/Users/buckycheng/Jupyter/Huge_Stock/10Q'

senti_path_10Q = '/Users/buckycheng/Jupyter/Huge_Stock/Sentiments/10Q'

for i in range(len(df_trickers)):
    cik=df_trickers['CIK'].iloc[i]
    ticker=df_trickers['Company'].iloc[i]
    CIK_Folder = path+'/'+str(cik)+'/clean_data'
    os.chdir(CIK_Folder)
    file_list = [fname for fname in os.listdir()]
    for cik_file in file_list:
        fname = cik_file.split('.')[0]
        fdate = fname.split('_')[1]
        ftype = '10-Q'
        file_path=cik_file
        
        senti_list = data_sentiment(CIK_Folder, file_path, cik, ticker, fdate, ftype)
        result_df = pd.DataFrame(senti_list)
       # headers are CIK	CONAME	FDATE	FORM	SECFNAME	neg_score	neu_score	pos_score
        result_df.to_csv(senti_path_10Q + '/' +str(cik)+'_'+'10-Q_sentiment'+'.csv',encoding='utf-8', mode = 'a', index=False, header=False)
        print(str(cik)+' finished')

Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/66740/clean_data/66740_2011-08-04.txt
get scores
{'CIK': 66740, 'CONAME': '3M Company', 'FDATE': '2011-08-04', 'FORM': '10-Q', 'FileName': '66740_2011-08-04.txt', 'neg_score': 0.26604545454545464, 'neu_score': 0.3716818181818182, 'pos_score': 0.36231818181818176}
66740 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/66740/clean_data/66740_2013-10-31.txt
get scores
{'CIK': 66740, 'CONAME': '3M Company', 'FDATE': '2013-10-31', 'FORM': '10-Q', 'FileName': '66740_2013-10-31.txt', 'neg_score': 0.2747727272727272, 'neu_score': 0.36268181818181816, 'pos_score': 0.3624545454545455}
66740 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/66740/clean_data/66740_2017-08-01.txt
get scores
{'CIK': 66740, 'CONAME': '3M Company', 'FDATE': '2017-08-01', 'FORM': '10-Q', 'FileName': '66740_2017-08-01.txt', 'neg_score': 0.2655416666666667, 'neu_score': 0.3689166666666666, 'pos_score': 0.3654583333333333}
66740 finished
Now File: /Us

get scores
{'CIK': 66740, 'CONAME': '3M Company', 'FDATE': '2008-05-02', 'FORM': '10-Q', 'FileName': '66740_2008-05-02.txt', 'neg_score': 0.26775000000000004, 'neu_score': 0.3652500000000001, 'pos_score': 0.3670833333333334}
66740 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/66740/clean_data/66740_2017-10-31.txt
get scores
{'CIK': 66740, 'CONAME': '3M Company', 'FDATE': '2017-10-31', 'FORM': '10-Q', 'FileName': '66740_2017-10-31.txt', 'neg_score': 0.268125, 'neu_score': 0.36991666666666667, 'pos_score': 0.36204166666666665}
66740 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/66740/clean_data/66740_2013-08-01.txt
get scores
{'CIK': 66740, 'CONAME': '3M Company', 'FDATE': '2013-08-01', 'FORM': '10-Q', 'FileName': '66740_2013-08-01.txt', 'neg_score': 0.2704545454545455, 'neu_score': 0.36836363636363634, 'pos_score': 0.3611818181818182}
66740 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/66740/clean_data/66740_2015-07-30.txt
get scores
{'CIK': 667

get scores
{'CIK': 1800, 'CONAME': 'Abbott Laboratories', 'FDATE': '2009-05-05', 'FORM': '10-Q', 'FileName': '1800_2009-05-05.txt', 'neg_score': 0.2598, 'neu_score': 0.38180000000000003, 'pos_score': 0.35860000000000003}
1800 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/1800/clean_data/1800_2011-11-04.txt
get scores
{'CIK': 1800, 'CONAME': 'Abbott Laboratories', 'FDATE': '2011-11-04', 'FORM': '10-Q', 'FileName': '1800_2011-11-04.txt', 'neg_score': 0.2695, 'neu_score': 0.37199999999999994, 'pos_score': 0.35833333333333334}
1800 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/1800/clean_data/1800_2012-08-07.txt
get scores
{'CIK': 1800, 'CONAME': 'Abbott Laboratories', 'FDATE': '2012-08-07', 'FORM': '10-Q', 'FileName': '1800_2012-08-07.txt', 'neg_score': 0.2654, 'neu_score': 0.378, 'pos_score': 0.35660000000000003}
1800 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/1800/clean_data/1800_2010-05-04.txt
get scores
{'CIK': 1800, 'CONAME': 'Abbott Labor

get scores
{'CIK': 1551152, 'CONAME': 'AbbVie Inc.', 'FDATE': '2019-08-05', 'FORM': '10-Q', 'FileName': '1551152_2019-08-05.txt', 'neg_score': 0.2091, 'neu_score': 0.49529999999999996, 'pos_score': 0.2954}
1551152 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/1551152/clean_data/1551152_2017-11-07.txt
get scores
{'CIK': 1551152, 'CONAME': 'AbbVie Inc.', 'FDATE': '2017-11-07', 'FORM': '10-Q', 'FileName': '1551152_2017-11-07.txt', 'neg_score': 0.2683333333333333, 'neu_score': 0.38633333333333336, 'pos_score': 0.3456666666666667}
1551152 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/1551152/clean_data/1551152_2013-05-08.txt
get scores
{'CIK': 1551152, 'CONAME': 'AbbVie Inc.', 'FDATE': '2013-05-08', 'FORM': '10-Q', 'FileName': '1551152_2013-05-08.txt', 'neg_score': 0.2731428571428572, 'neu_score': 0.36500000000000005, 'pos_score': 0.36185714285714293}
1551152 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/1551152/clean_data/1551152_2016-05-06.txt
get

get scores
{'CIK': 815094, 'CONAME': 'ABIOMED Inc', 'FDATE': '2020-02-06', 'FORM': '10-Q', 'FileName': '815094_2020-02-06.txt', 'neg_score': 0.24483333333333335, 'neu_score': 0.4109166666666666, 'pos_score': 0.3441666666666667}
815094 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/815094/clean_data/815094_2008-11-10.txt
get scores
{'CIK': 815094, 'CONAME': 'ABIOMED Inc', 'FDATE': '2008-11-10', 'FORM': '10-Q', 'FileName': '815094_2008-11-10.txt', 'neg_score': 0.27316666666666667, 'neu_score': 0.3418333333333334, 'pos_score': 0.3851666666666667}
815094 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/815094/clean_data/815094_2012-08-06.txt
get scores
{'CIK': 815094, 'CONAME': 'ABIOMED Inc', 'FDATE': '2012-08-06', 'FORM': '10-Q', 'FileName': '815094_2012-08-06.txt', 'neg_score': 0.2582, 'neu_score': 0.3684, 'pos_score': 0.3734}
815094 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/815094/clean_data/815094_2011-11-07.txt
get scores
{'CIK': 815094, 'CONA

...

get scores
{'CIK': 320193, 'CONAME': 'Apple Inc.', 'FDATE': '2019-05-01', 'FORM': '10-Q', 'FileName': '320193_2019-05-01.txt', 'neg_score': 0.2747857142857143, 'neu_score': 0.37228571428571433, 'pos_score': 0.35300000000000004}
320193 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/320193/clean_data/320193_2011-07-20.txt
get scores
{'CIK': 320193, 'CONAME': 'Apple Inc.', 'FDATE': '2011-07-20', 'FORM': '10-Q', 'FileName': '320193_2011-07-20.txt', 'neg_score': 0.2933125, 'neu_score': 0.3113125, 'pos_score': 0.395375}
320193 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/320193/clean_data/320193_2015-01-28.txt
get scores
{'CIK': 320193, 'CONAME': 'Apple Inc.', 'FDATE': '2015-01-28', 'FORM': '10-Q', 'FileName': '320193_2015-01-28.txt', 'neg_score': 0.29240000000000005, 'neu_score': 0.32020000000000004, 'pos_score': 0.38706666666666667}
320193 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/320193/clean_data/320193_2020-01-29.txt
get scores
{'CIK': 32019

get scores
{'CIK': 6951, 'CONAME': 'Applied Materials Inc.', 'FDATE': '2017-05-25', 'FORM': '10-Q', 'FileName': '6951_2017-05-25.txt', 'neg_score': 0.2855714285714286, 'neu_score': 0.3458571428571428, 'pos_score': 0.36864285714285716}
6951 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/6951/clean_data/6951_2013-08-22.txt
get scores
{'CIK': 6951, 'CONAME': 'Applied Materials Inc.', 'FDATE': '2013-08-22', 'FORM': '10-Q', 'FileName': '6951_2013-08-22.txt', 'neg_score': 0.2802, 'neu_score': 0.3538666666666667, 'pos_score': 0.3658666666666666}
6951 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/6951/clean_data/6951_2011-02-28.txt
get scores
{'CIK': 6951, 'CONAME': 'Applied Materials Inc.', 'FDATE': '2011-02-28', 'FORM': '10-Q', 'FileName': '6951_2011-02-28.txt', 'neg_score': 0.2765333333333333, 'neu_score': 0.36006666666666665, 'pos_score': 0.3634}
6951 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/6951/clean_data/6951_2014-08-21.txt
get scores
{'CIK'

get scores
{'CIK': 6951, 'CONAME': 'Applied Materials Inc.', 'FDATE': '2017-02-23', 'FORM': '10-Q', 'FileName': '6951_2017-02-23.txt', 'neg_score': 0.28471428571428564, 'neu_score': 0.35021428571428576, 'pos_score': 0.3649285714285715}
6951 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/6951/clean_data/6951_2010-06-09.txt
get scores
{'CIK': 6951, 'CONAME': 'Applied Materials Inc.', 'FDATE': '2010-06-09', 'FORM': '10-Q', 'FileName': '6951_2010-06-09.txt', 'neg_score': 0.2755625, 'neu_score': 0.3610625, 'pos_score': 0.363375}
6951 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/6951/clean_data/6951_2019-05-23.txt
get scores
{'CIK': 6951, 'CONAME': 'Applied Materials Inc.', 'FDATE': '2019-05-23', 'FORM': '10-Q', 'FileName': '6951_2019-05-23.txt', 'neg_score': 0.2844, 'neu_score': 0.34346666666666664, 'pos_score': 0.3721333333333333}
6951 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/6951/clean_data/6951_2009-03-03.txt
get scores
{'CIK': 6951, 'CONAME

get scores
{'CIK': 1521332, 'CONAME': 'Aptiv PLC', 'FDATE': '2015-04-30', 'FORM': '10-Q', 'FileName': '1521332_2015-04-30.txt', 'neg_score': 0.2852941176470588, 'neu_score': 0.3660588235294117, 'pos_score': 0.34858823529411764}
1521332 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/1521332/clean_data/1521332_2019-10-30.txt
get scores
{'CIK': 1521332, 'CONAME': 'Aptiv PLC', 'FDATE': '2019-10-30', 'FORM': '10-Q', 'FileName': '1521332_2019-10-30.txt', 'neg_score': 0.24813636363636365, 'neu_score': 0.4356818181818182, 'pos_score': 0.31613636363636366}
1521332 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/1521332/clean_data/1521332_2014-10-24.txt
get scores
{'CIK': 1521332, 'CONAME': 'Aptiv PLC', 'FDATE': '2014-10-24', 'FORM': '10-Q', 'FileName': '1521332_2014-10-24.txt', 'neg_score': 0.28240000000000004, 'neu_score': 0.3824666666666667, 'pos_score': 0.33506666666666673}
1521332 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/1521332/clean_data/1521332

get scores
{'CIK': 7084, 'CONAME': 'Archer-Daniels-Midland Co', 'FDATE': '2011-11-07', 'FORM': '10-Q', 'FileName': '7084_2011-11-07.txt', 'neg_score': 0.27714285714285714, 'neu_score': 0.38128571428571434, 'pos_score': 0.34128571428571425}
7084 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/7084/clean_data/7084_2008-11-10.txt
get scores
{'CIK': 7084, 'CONAME': 'Archer-Daniels-Midland Co', 'FDATE': '2008-11-10', 'FORM': '10-Q', 'FileName': '7084_2008-11-10.txt', 'neg_score': 0.2702, 'neu_score': 0.3836, 'pos_score': 0.34619999999999995}
7084 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/7084/clean_data/7084_2010-05-10.txt
get scores
{'CIK': 7084, 'CONAME': 'Archer-Daniels-Midland Co', 'FDATE': '2010-05-10', 'FORM': '10-Q', 'FileName': '7084_2010-05-10.txt', 'neg_score': 0.2855, 'neu_score': 0.363125, 'pos_score': 0.351125}
7084 finished
Now File: /Users/buckycheng/Jupyter/Huge_Stock/10Q/7084/clean_data/7084_2018-07-31.txt
get scores
{'CIK': 7084, 'CONAME': 'Ar